In [1]:
# import importlib
# import db_utils
# importlib.reload(db_utils)

In [2]:
%pip install langchain-elasticsearch langchain-community langchain_experimental langchain_openai tqdm pypdf streamlit langchain-ollama --use-pep517

Note: you may need to restart the kernel to use updated packages.


In [3]:
from langchain_community.embeddings import FastEmbedEmbeddings
from langchain_community.llms.ollama import Ollama

from db_utils import add_documents_to_db, remove_index, remove_document, fetch_all
from files_utils import chunk_documents
from model_utils import rag_prompt
from config import MODEL_NAME, ES_PORT, ES_INDEX_NAME, ES_DISTANCE_STRATEGY, CONTEXT_CHAR_THRESHOLD, CHUNKER_TYPE
import pickle

Prepare chunks

In [4]:
pdf_directory = "./data/college"

document_chunks = chunk_documents(pdf_directory=pdf_directory, chunker_type=CHUNKER_TYPE)

Loaded 13 documents.


In [5]:
document_chunks

[Document(metadata={'source': 'data\\college\\CIT-8.pdf', 'page': 0, 'id': 'data\\college\\CIT-8.pdf:0:0'}, page_content='Deklarację CIT -8 składa się w przypadku:  \n• Podmiotów podlegających opodatkowaniu podatkiem dochodowym od osób \nprawnych , takich jak:  \no Spółki kapitałowe  (spółka z o.o., spółka akcyjna).  \no Spółdzielnie, przedsiębiorstwa państwowe, jednostki organizacyjne \nniemające osobowości prawnej , które są podatnikami CIT.'),
 Document(metadata={'source': 'data\\college\\CIT-8.pdf', 'page': 0, 'id': 'data\\college\\CIT-8.pdf:0:1'}, page_content='o Spółki kapitałowe  (spółka z o.o., spółka akcyjna).  \no Spółdzielnie, przedsiębiorstwa państwowe, jednostki organizacyjne \nniemające osobowości prawnej , które są podatnikami CIT.  \n• Organizacje pozarządowe , fundacje i stowarzyszenia, jeśli uzyskują przychody \npodlegające opodatkowaniu.  \nDeklaracji CIT -8 nie składają:  \n• Podmioty zwolnione z podatku dochodowego , np. niektóre organizacje'),
 Document(metadata={

In [6]:
len(document_chunks)

62

In [7]:
# save document_chunks to a file
with open("document_chunks_FastEmbedEmbeddings_college.pkl", "wb") as f:
    pickle.dump(document_chunks, f)

In [8]:
# %pip install langchain-elasticsearch langchain-community langchain_experimental langchain_openai tqdm pypdf streamlit fastembed --use-pep517

Prepare database

In [13]:
from langchain_ollama import OllamaEmbeddings

embedding = OllamaEmbeddings(
    model="Lexi-Llama-3-8B-Uncensored_Q8_0.gguf",
)
db_kwargs = {
    "embedding": embedding,
    "es_url": "http://localhost:9200",
    "index_name": "rag",
    "distance_strategy": "COSINE"
}

In [14]:
# clear index 
remove_index(index_name=db_kwargs["index_name"], db_config={"hosts": db_kwargs["es_url"]})

Index 'rag' does not exist.


In [15]:
#print type of document_chunks, db_kwargs
print(type(document_chunks), type(db_kwargs))

<class 'list'> <class 'dict'>


In [16]:
db = add_documents_to_db(db=None, document_chunks=document_chunks, db_kwargs=db_kwargs, bulk_upload=True)

Bulk ingesting documents...


ResponseError: model "Lexi-Llama-3-8B-Uncensored_Q8_0" not found, try pulling it first

In [ ]:
fetch_all(
    index_name=db_kwargs["index_name"],
    db_config={"hosts": db_kwargs["es_url"]},
)

Prompt rag

In [ ]:
model = Ollama(model=MODEL_NAME)

response = rag_prompt(
    context_char_threshold=CONTEXT_CHAR_THRESHOLD,
    query = "Jaki wydział jest potocznie nazywany CPN -em?",
    model=model,
    db=db,
    chunker_type=CHUNKER_TYPE,
)

# remove_document(
#     index_name=db_kwargs["index_name"],
#     source_file="",
#     db_config={"hosts": db_kwargs["es_url"]},
#     )
